In [1]:
%run Archpool.ipynb
%run Argparser.ipynb
%run Topo_treatment.ipynb
%run Net.ipynb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image

import model
from dataset import *
from utils import check_dir
from tqdm import tqdm
import numpy as np
from functools import reduce

import os
import json
import argparse

import sys
import json
sys.path.insert(0, './persis_lib_cpp')
from persis_homo_optimal import *

torch_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def parse_args():
    '''parser = argparse.ArgumentParser()
    parser.add_argument('config', help='configuration file path')
    opt = parser.parse_args()
    with open(opt.config, 'r') as inf:
        config = json.load(inf)'''
    
    f = open('example.json')
    config = json.load(f)

    try:
        if config['output_path'][-1] != '/':
            config['output_path'] += '/'
        if config['train_data_path'][-1] != '/':
            config['train_data_path'] += '/'
        if config['val_data_path'][-1] != '/':
            config['val_data_path'] += '/'
    except KeyError as err:
        print(f'{opt.config}: Unspecified path {err}')
        exit(1)
    return config


def initialize_network(config):
    network = {}
    random_seed = 0
    try:
        config['resume']
        random_seed = config['random_seed']
    except KeyError:
        config['resume'] = False
    torch.manual_seed(random_seed)
    if (config['resume'] and os.path.isfile(config['resume'])):
        confout = config['output_path'] + config['name'] + '/'
        network['resume'] = True
        checkpoint = torch.load(config['resume'], map_location=torch_device)
        network['epoch_start'] = checkpoint['epoch'] + \
            1 if checkpoint['output_dir'] == confout else 0
        network['epoch_end'] = config['epoch'] or checkpoint['epoch_end']
        network['output_dir'] = confout
        network['checkpoint_dir'] = checkpoint['checkpoint_dir']
        network['learning_rate'] = checkpoint['learning_rate']
        network['train_data_dir'] = checkpoint['train_data_dir']
        network['val_data_dir'] = checkpoint['val_data_dir']
        network['name'] = checkpoint['name']
        network['batch_size'] = checkpoint['batch_size']
        network['features'] = checkpoint['features']
        network['image_size'] = checkpoint['image_size']
        network['image_channels'] = checkpoint['image_channels']
        network['optimizer_name'] = checkpoint['optimizer_name']
        network['arch'] = checkpoint['arch']
        network['bn'] = checkpoint['bn']
        network['checkpoint'] = checkpoint
    else:
        network['resume'] = False
        try:
            network['output_dir'] = config['output_path'] + \
                config['name'] + '/'
            network['name'] = config['name']
            network['epoch_start'] = 0
            network['epoch_end'] = config['epoch']
            network['learning_rate'] = config['learning_rate']
            network['batch_size'] = config['batch_size']
            network['features'] = int(config['features'])
            network['image_size'] = config['image_size']
            network['image_channels'] = config['image_channels']
            network['optimizer_name'] = config['optimizer']
            network['train_data_dir'] = config['train_data_path']
            network['val_data_dir'] = config['val_data_path']
            network['arch'] = config['arch']
            network['bn'] = config['bn']
        except KeyError as err:
            print(f'Configuration: Unspecified field {err}')
            exit(1)
    network['checkpoint_dir'] = network['output_dir'] + 'checkpoints/'
    network['result_dir'] = network['output_dir'] + 'result/'
    check_dir(network['output_dir'])
    check_dir(network['checkpoint_dir'])
    check_dir(network['result_dir'])

    network['logfile_path'] = network['result_dir'] + 'logfile.txt'
    network['performance_path'] = network['result_dir'] + 'performance.txt'
    learning_model = model.AutoEncoder(
        network['image_size'], network['image_channels'], network['features'], network['arch'], network['bn'])
    #learning_model.load_state_dict(torch.load("C:/Users/nelsite/Desktop/Coding_with_Fahim/Topological_Segmentation/TopoSegNetSimple/network/Output/Unet_Training/checkpoints/Unet_Training_49.pth"))
    learning_model = learning_model.to(torch_device)
    
    network['loss_function'] = WeightedBCELoss(one_weight=1,zeros_weight=1)
    # network['loss_function'] = nn.MSELoss()
    optimizer = None
    if network['optimizer_name'] == 'adam':
        optimizer = optim.Adam(learning_model.parameters(),
                               lr=network['learning_rate'])
    elif network['optimizer_name'] == 'sgd':
        optimizer = optim.SGD(learning_model.parameters(),
                              momentum=0.9, weight_decay=1e-2,
                              lr=network['learning_rate'])

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=1)

    if (network['resume']):
        learning_model.load_state_dict(network['checkpoint']['model'])
        optimizer.load_state_dict(network['checkpoint']['optimizer'])
        scheduler.load_state_dict(network['checkpoint']['scheduler'])
    
    network['model'] = learning_model
    network['optimizer'] = optimizer
    network['scheduler'] = scheduler
    return network


class WeightedBCELoss:
    def __init__(self, one_weight=1.0, zeros_weight=1.0, reduction="mean"):
        self.reduction = reduction
        self.update_weights(one_weight, zeros_weight)

    def update_weights(self, one_weight, zeros_weight):
        self.weights = torch.FloatTensor([one_weight, zeros_weight])
        self.weights.to(torch_device)

    def _bce(self, x, y):
        weights = -self.weights
        x = torch.clamp(x, min=1e-7, max=1-1e-7)
        y = torch.clamp(y, min=1e-7, max=1-1e-7)
        return weights[1]*y*torch.log(x) + weights[0]*(1-y)*torch.log(1-x)

    def __call__(self, pred, truth):
        loss = self._bce(pred, truth)
        if self.reduction == 'mean':
            return torch.mean(loss)
        if self.reduction == 'sum':
            return torch.sum(loss)
        return loss



def train(adv_params, network, dataloader, withTopo=True):
    
    et = Edges_(adv_params, debug=False)
    #criterionT = GANLoss("vanilla_topo", "sum").to(torch_device)
    criterionT = nn.BCEWithLogitsLoss(reduction="sum")
    loss_function = network['loss_function']
    model = network['model']
    optimizer = network['optimizer']
    model.train()
    running_loss = 0.0
    t_loss = 0.0
    alpha = 0.96
    result_dir = network['result_dir']
    batch_number = 0
    step_num = 0
    tot_append_loss = 0.0
    for data in dataloader:
        
        step_num = step_num + 1
        print("step_num : ", step_num)
        scalars, label = data

        scalars = scalars.to(torch_device)
        label = label.to(torch_device)
        optimizer.zero_grad()

        p = model(scalars)
        prediction = (p - torch.min(p))/(torch.max(p) - torch.min(p)) *2-1
        
        loss = loss_function(prediction, label)
        running_loss += loss.item()
        print("training loss : ", loss.item())
        
        #loss.backward()
        num_rows = p.size(0)
        s = scalars.cpu().view(
            num_rows, 1, 64, 64).double()
        t = label.cpu().view(
            num_rows, 1, 64, 64).double()

        pred = prediction.cpu().view(
            num_rows, 1, 64, 64).double()
        
        pl = label.cpu().view(
            num_rows, 1, 64, 64).double()
        
        out_image = torch.transpose(torch.stack((s, t, pl,pred)), 0, 1).reshape(
            4*num_rows, 1,  64, 64)
        save_image(out_image.cpu(
        ), f"{result_dir}_batch{batch_number}.png", padding=4, nrow=24)
        batch_number += 1
        
        if withTopo:
            
            tp_wgt   = et.return_tp_weight()
            fake_fix, mean_wasdis = et.fix_with_topo(prediction.detach().cpu().numpy(), label.detach().cpu().numpy(),
                                                     et.return_target_dim(), result_dir, batch_number, num_rows, 
                                                     -1.0, 1.0, blind=et.blind())
            #print( prediction.detach().cpu().numpy().shape, et.return_target_dim())
            fake_fix = torch.from_numpy(fake_fix).to(torch_device)
            
            fake_fix = torch.unsqueeze(fake_fix, 1)
            fake_fix = (fake_fix - torch.min(fake_fix))/(torch.max(fake_fix) - torch.min(fake_fix))
            errT = criterionT(p, fake_fix) * tp_wgt
            t_loss += errT.item()
            print("Topo loss : ", errT.item())
            #errT.backward()
            #total_loss = (alpha * loss) + ((1-alpha) * errT)
            tot_loss = loss + errT
            tot_append_loss += tot_loss.item()
            print("Total loss : ", tot_loss.item())
            tot_loss.backward()
            optimizer.step()
        
        else:
            #loss.backward()
            optimizer.step()
    
    training_loss = running_loss / len(dataloader.dataset)
    topo_loss = t_loss / len(dataloader.dataset)
    total_loss = tot_append_loss / len(dataloader.dataset)
    
    return [training_loss], [topo_loss], [total_loss]


def validate(network, dataloader, epoch):
    image_size = [network['image_size'], network['image_size']]
    running_loss = 0.0
    tp = 0.0  # true positive
    tn = 0.0  # true negative
    fp = 0.0  # false positive
    fn = 0.0  # false negative

    l1_diff = 0.0
    with torch.no_grad():
        loss_function = network['loss_function']
        model = network['model']
        result_dir = network['result_dir']
        image_channels = network['image_channels']
        model.eval()
        batch_number = 0
        output_image = False
        for i, data in enumerate(dataloader):
            scalars, label = data
            label = label.to(torch_device)
            scalars = scalars.to(torch_device)
            batch_size = label.size(0)

            prediction = model(scalars)
            '''pred = prediction.cpu().detach().numpy()
            pred = np.squeeze(pred)
            fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(12, 4))
            for i in range(5):
                ax[i].imshow(pred[i], cmap='gray')
                ax[i].axis('off')

            plt.show()'''
            
            loss = loss_function(prediction, label)
            running_loss += loss.item() * batch_size
            # log accuracy
            pred = prediction.cpu().view(batch_size, -1).double()
            truth = label.cpu().view(batch_size, -1).double()
            truth[]

            plabel = torch.zeros(pred.size())
            plabel[pred >= 0.3] = 1
            
            tp += torch.sum(torch.logical_and(plabel == 1, truth == 1).float())
            tn += torch.sum(torch.logical_and(plabel == 0, truth == 0).float())
            fp += torch.sum(torch.logical_and(plabel == 1, truth == 0).float())
            fn += torch.sum(torch.logical_and(plabel == 0, truth == 1).float())

            l1_diff += torch.sum(torch.abs(pred - truth))

            if epoch != "":
                if (epoch == network['epoch_end'] - 1) or (i == len(dataloader) - 1):
                    output_image = True

            #if output_image:
            num_rows = batch_size
            s = scalars.cpu().view(
                num_rows, 1, image_size[1], image_size[0]).double()
            t = label.cpu().view(
                num_rows, 1, image_size[1], image_size[0]).double()

            pred = prediction.cpu().view(
                num_rows, 1, image_size[1], image_size[0]).double()
            
            pl = plabel.cpu().view(
                num_rows, 1, image_size[1], image_size[0]).double()
            
            out_image = torch.transpose(torch.stack((s, t, pl,pred)), 0, 1).reshape(
                4*num_rows, 1,  image_size[1], image_size[0])
            save_image(out_image.cpu(
            ), f"{result_dir}epoch_{epoch}_batch{batch_number}.png", padding=4, nrow=24)
            batch_number += 1
        # end for loop
    # end with nograd
    val_loss = running_loss/len(dataloader.dataset)
    l1_diff /= len(dataloader.dataset)
    tp /= len(dataloader.dataset)
    tn /= len(dataloader.dataset)
    fp /= len(dataloader.dataset)
    fn /= len(dataloader.dataset)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = (tp)/(tp+fp)
    recall = (tp)/(tp+fn)
    f1 = 2*tp / (2 * tp + fp + fn)

    return [val_loss], [accuracy, precision, recall, f1, l1_diff]


def floats2str(l):
    return ",".join(map(lambda x: f'{x:.6f}', l))


def parameters_count(model):
    total = 0
    total_t = 0
    for p in model.parameters():
        if p.requires_grad:
            total += p.numel()
            total_t += p.numel()
        else:
            total += p.numel()
    return total, total_t


def main():
    adv_params  = return_advanced_params()
    config = parse_args()
    network = initialize_network(config)

    p, pt = parameters_count(network['model'])
    print(f'number of parameters(trainable) {p}({pt})')

    with open(network['output_dir']+'config.json', 'w') as jsonout:
        json.dump(config, jsonout, indent=2)

    train_dataset = ImageBoundary(
        config['train_data_path'], network['image_channels'])
    #i,t = train_dataset[0]
    #print(i,t)
    train_dataloader = DataLoader(
        train_dataset, batch_size=network['batch_size'], shuffle=True)
    val_dataset = ImageBoundary(
        config['val_data_path'], network['image_channels'])
    val_dataloader = DataLoader(
        val_dataset, batch_size=network['batch_size'], shuffle=False)

    '''if network['resume']:
        logfile = open(network['logfile_path'], 'a')
        perf_log = open(network['performance_path'], 'a')
    else:'''
    logfile = open(network['logfile_path'], 'w')
    logfile.write('epoch,train_loss,val_loss,topo_loss,total_loss\n')
    perf_log = open(network['performance_path'], 'w')
    perf_log.write(
        'epoch, accuracy, precision, recall, f1, l1_diff_per_image)\n')

    for epoch in tqdm(range(network['epoch_start'], network['epoch_end'])):
        t_loss, topo_loss, total_loss = train(adv_params, network, train_dataloader, True)
        v_loss, performance = validate(network, val_dataloader, epoch)
        network['scheduler'].step(total_loss[0])

        performance = floats2str(performance)
        perf_log.write(f'{epoch},{performance}\n')
        perf_log.flush()

        t_loss = floats2str(t_loss)
        v_loss = floats2str(v_loss)
        topo_loss = floats2str(topo_loss)
        total_loss = floats2str(total_loss)
        
        logfile.write(f'{epoch},{t_loss},{v_loss},{topo_loss},{total_loss}\n')
        logfile.flush()
        if ((epoch+1) % 50 == 0) or epoch == network['epoch_end'] - 1:
            torch.save({
                'epoch': epoch,
                'epoch_end': network['epoch_end'],
                'model': network['model'].state_dict(),
                'optimizer': network['optimizer'].state_dict(),
                'optimizer_name': network['optimizer_name'],
                'scheduler': network['scheduler'].state_dict(),
                'checkpoint_dir': network['checkpoint_dir'],
                'train_data_dir': network['train_data_dir'],
                'val_data_dir': network['val_data_dir'],
                'output_dir': network['output_dir'],
                'name': network['name'],
                'batch_size': network['batch_size'],
                'learning_rate': network['learning_rate'],
                'features': network['features'],
                'image_size': network['image_size'],
                'image_channels': network['image_channels'],
                'arch': network['arch'],
                'bn': network['bn']
            }, f'{network["checkpoint_dir"]}{network["name"]}_{epoch}.pth')
    logfile.close()
    perf_log.close()


if __name__ == '__main__':
    main()

number of parameters(trainable) 7762465(7762465)


  0%|                                                                                           | 0/30 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\envs\topo_gan\lib\site-packages\ipykernel_launcher.py:504: DeprecationWarning: Call to deprecated class PersImage. (Replaced with the class `persim.PersistenceImager`.) -- Deprecated since version 0.1.5.


step_num :  1
training loss :  0.12208235263824463
Computing 1D 1-wasserstein distance.
Topo loss :  6.768273830413818
Total loss :  6.890356063842773
step_num :  2
training loss :  0.13557069003582
Computing 1D 1-wasserstein distance.
Topo loss :  6.744208335876465
Total loss :  6.879778861999512
step_num :  3
training loss :  0.09960757941007614
Computing 1D 1-wasserstein distance.
Topo loss :  6.773429870605469
Total loss :  6.873037338256836
step_num :  4
training loss :  0.09813620150089264
Computing 1D 1-wasserstein distance.
Topo loss :  6.711061954498291
Total loss :  6.809198379516602
step_num :  5
training loss :  0.09451078623533249
Computing 1D 1-wasserstein distance.
Topo loss :  6.725515842437744
Total loss :  6.820026397705078
step_num :  6
training loss :  0.11855163425207138
Computing 1D 1-wasserstein distance.
Topo loss :  6.74755859375
Total loss :  6.866110324859619
step_num :  7
training loss :  0.13089627027511597
Computing 1D 1-wasserstein distance.
Topo loss :  

Computing 1D 1-wasserstein distance.
Topo loss :  6.869258403778076
Total loss :  7.975317001342773
step_num :  57
training loss :  1.1843847036361694
Computing 1D 1-wasserstein distance.
Topo loss :  6.8447771072387695
Total loss :  8.02916145324707
step_num :  58
training loss :  1.0555304288864136
Computing 1D 1-wasserstein distance.
Topo loss :  6.844009876251221
Total loss :  7.899540424346924
step_num :  59
training loss :  1.0982099771499634
Computing 1D 1-wasserstein distance.
Topo loss :  6.9225640296936035
Total loss :  8.020773887634277
step_num :  60
training loss :  1.2667044401168823
Computing 1D 1-wasserstein distance.
Topo loss :  6.904822826385498
Total loss :  8.171526908874512
step_num :  61
training loss :  1.220495581626892
Computing 1D 1-wasserstein distance.
Topo loss :  6.891338348388672
Total loss :  8.111833572387695
step_num :  62
training loss :  1.0883455276489258
Computing 1D 1-wasserstein distance.
Topo loss :  6.886018753051758
Total loss :  7.9743642807

Computing 1D 1-wasserstein distance.
Topo loss :  6.9367876052856445
Total loss :  7.981164455413818
step_num :  112
training loss :  1.2282960414886475
Computing 1D 1-wasserstein distance.
Topo loss :  6.944833278656006
Total loss :  8.173129081726074
step_num :  113
training loss :  1.184040904045105
Computing 1D 1-wasserstein distance.
Topo loss :  6.962271690368652
Total loss :  8.146312713623047
step_num :  114
training loss :  1.0277942419052124
Computing 1D 1-wasserstein distance.
Topo loss :  6.931698799133301
Total loss :  7.959493160247803
step_num :  115
training loss :  1.3331598043441772
Computing 1D 1-wasserstein distance.
Topo loss :  6.924233913421631
Total loss :  8.257393836975098
step_num :  116
training loss :  1.1400063037872314
Computing 1D 1-wasserstein distance.
Topo loss :  6.984057903289795
Total loss :  8.124064445495605
step_num :  117
training loss :  1.0622628927230835
Computing 1D 1-wasserstein distance.
Topo loss :  6.946583271026611
Total loss :  8.0088

...

Computing 1D 1-wasserstein distance.
Topo loss :  6.811919689178467
Total loss :  7.774510383605957
step_num :  219
training loss :  0.843845546245575
Computing 1D 1-wasserstein distance.
Topo loss :  6.838746547698975
Total loss :  7.682591915130615
step_num :  220
training loss :  0.9912975430488586
Computing 1D 1-wasserstein distance.
Topo loss :  6.780832290649414
Total loss :  7.772130012512207
step_num :  221
training loss :  0.8775508999824524
Computing 1D 1-wasserstein distance.
Topo loss :  6.846848011016846
Total loss :  7.724399089813232
step_num :  222
training loss :  0.9715753793716431
Computing 1D 1-wasserstein distance.
Topo loss :  6.771199703216553
Total loss :  7.742774963378906
step_num :  223
training loss :  0.9191469550132751
Computing 1D 1-wasserstein distance.
Topo loss :  6.827109336853027
Total loss :  7.746256351470947
step_num :  224
training loss :  0.9010797739028931
Computing 1D 1-wasserstein distance.
Topo loss :  6.815518856048584
Total loss :  7.71659

Computing 1D 1-wasserstein distance.
Topo loss :  6.773150444030762
Total loss :  7.612745761871338
step_num :  273
training loss :  0.7511446475982666
Computing 1D 1-wasserstein distance.
Topo loss :  6.790108680725098
Total loss :  7.541253089904785
step_num :  274
training loss :  0.8428387641906738
Computing 1D 1-wasserstein distance.
Topo loss :  6.812087059020996
Total loss :  7.65492582321167
step_num :  275
training loss :  0.9099050760269165
Computing 1D 1-wasserstein distance.
Topo loss :  6.831791400909424
Total loss :  7.741696357727051
step_num :  276
training loss :  0.917493999004364
Computing 1D 1-wasserstein distance.
Topo loss :  6.813349723815918
Total loss :  7.730843544006348
step_num :  277
training loss :  0.8549043536186218
Computing 1D 1-wasserstein distance.
Topo loss :  6.850058078765869
Total loss :  7.704962253570557
step_num :  278
training loss :  0.894740104675293
Computing 1D 1-wasserstein distance.
Topo loss :  6.823042869567871
Total loss :  7.7177829

Computing 1D 1-wasserstein distance.
Topo loss :  6.875202178955078
Total loss :  7.738375663757324
step_num :  327
training loss :  0.9398307204246521
Computing 1D 1-wasserstein distance.
Topo loss :  6.8593549728393555
Total loss :  7.799185752868652
step_num :  328
training loss :  0.9691073298454285
Computing 1D 1-wasserstein distance.
Topo loss :  6.780113697052002
Total loss :  7.749220848083496
step_num :  329
training loss :  0.9001690149307251
Computing 1D 1-wasserstein distance.
Topo loss :  6.834197998046875
Total loss :  7.7343668937683105
step_num :  330
training loss :  0.9026510119438171
Computing 1D 1-wasserstein distance.
Topo loss :  6.809057235717773
Total loss :  7.711708068847656
step_num :  331
training loss :  0.9586801528930664
Computing 1D 1-wasserstein distance.
Topo loss :  6.816839218139648
Total loss :  7.775519371032715
step_num :  332
training loss :  0.9555191993713379
Computing 1D 1-wasserstein distance.
Topo loss :  6.815431118011475
Total loss :  7.77

Computing 1D 1-wasserstein distance.
Topo loss :  6.786149024963379
Total loss :  7.543605804443359
step_num :  381
training loss :  0.7118780612945557
Computing 1D 1-wasserstein distance.
Topo loss :  6.806491374969482
Total loss :  7.518369674682617
step_num :  382
training loss :  0.983221173286438
Computing 1D 1-wasserstein distance.
Topo loss :  6.83026647567749
Total loss :  7.813487529754639
step_num :  383
training loss :  0.8276471495628357
Computing 1D 1-wasserstein distance.
Topo loss :  6.832136154174805
Total loss :  7.659783363342285
step_num :  384
training loss :  0.824313759803772
Computing 1D 1-wasserstein distance.
Topo loss :  6.813366413116455
Total loss :  7.6376800537109375
step_num :  385
training loss :  0.8989962935447693
Computing 1D 1-wasserstein distance.
Topo loss :  6.814637184143066
Total loss :  7.7136335372924805
step_num :  386
training loss :  0.9937209486961365
Computing 1D 1-wasserstein distance.
Topo loss :  6.809365749359131
Total loss :  7.80308

100%|███████████████████████████████████████████████████████████████████████████████| 30/30 [1:47:31<00:00, 215.06s/it]
